In [43]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
import os

In [46]:
def mean_squared_displacement_1(file_path):

    data = pd.read_csv(file_path, delim_whitespace=True, header=None, 
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    x_smooth = np.array(data['x_smooth'])
    y_smooth = np.array(data['y_smooth'])
    z_smooth = np.array(data['z_smooth'])
    time = np.array(data['time'])
    
    msd_by_tau = {}

    for tau in range(1, len(time)):
        squared_displacements = []
        tau_times = []

        for i in range(len(time) - tau):
            tau_time = np.round((time[i + tau] - time[i]), 3)
            tau_times.append(tau_time)

            squared_disp = ((x_smooth[i + tau] - x_smooth[i]) ** 2 +
                            (y_smooth[i + tau] - y_smooth[i]) ** 2 +
                            (z_smooth[i + tau] - z_smooth[i]) ** 2)
            squared_displacements.append(squared_disp)

        msd_by_tau[round(np.mean(tau_times), 3)] = np.mean(squared_displacements) 

    return msd_by_tau




In [47]:
file_path = ('FINALDATA/WT_planktonic_final/track10194_0 (2)_processed.txt')
msd_by_tau = mean_squared_displacement(file_path)

# Extract tau and MSD values for plotting
tau_values = list(msd_by_tau.keys())
msd_values = list(msd_by_tau.values())

# Apply natural log to both tau and MSD values
log_tau_values = np.log(tau_values)
log_msd_values = np.log(msd_values)

# Plot MSD vs Tau on natural logarithmic scales
plt.figure(figsize=(8, 6))
plt.scatter(log_tau_values, log_msd_values)

# Labels and title
plt.xlabel('ln(Tau) (seconds)')
plt.ylabel('ln(Mean Squared Displacement) (MSD)')
plt.title('MSD vs Tau (Natural Log Scale)')

# Add gridlines for easier interpretation
plt.grid(True, which='both')

# Display the plot
plt.show()

AttributeError: 'tuple' object has no attribute 'keys'

In [48]:
def mean_squared_displacement(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, header=None, 
                       names=['time', 'x', 'y', 'z', 'x_smooth', 'y_smooth', 'z_smooth'])
    x_smooth = np.array(data['x_smooth'])
    y_smooth = np.array(data['y_smooth'])
    z_smooth = np.array(data['z_smooth'])
    time = np.array(data['time'])
    
    tau_values = []
    msd_values = []

    for tau in range(1, len(time)):
        squared_displacements = []
        tau_times = []

        for i in range(len(time) - tau):
            tau_time = np.round((time[i + tau] - time[i]), 3)
            tau_times.append(tau_time)

            squared_disp = ((x_smooth[i + tau] - x_smooth[i]) ** 2 +
                            (y_smooth[i + tau] - y_smooth[i]) ** 2 +
                            (z_smooth[i + tau] - z_smooth[i]) ** 2)
            squared_displacements.append(squared_disp)

        tau_values.append(np.mean(tau_times))
        msd_values.append(np.mean(squared_displacements))

    return np.array(tau_values), np.array(msd_values)

    

In [26]:
from scipy.stats import linregress
def find_gradient_intercept(file_path):
    tau_arr , msd_arr  = mean_squared_displacement(file_path)
    
    log_tau = np.log(tau_arr)
    log_msd = np.log(msd_arr)
    
    slope, intercept = linregress(log_tau, log_msd)
    
    return slope , intercept
    

    

In [41]:
def process_directory_1(directory_path):
    gradients = []
    intercepts = []
    
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        
        if file_path.endswith('.txt'):
            slope, intercept = find_gradient_intercept(file_path)
            gradients.append(slope)
            intercepts.append(intercept)
    
    return np.array(gradients), np.array(intercepts)

In [36]:
from multiprocessing import Pool
def process_directory_2(directory_path):
    files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.txt')]
    
    # Use multiprocessing to parallelize file processing
    with Pool() as pool:
        results = pool.map(find_gradient_intercept, files)
    
    gradients, intercepts = zip(*results)
    return np.array(gradients), np.array(intercepts)


(1.7481764411255623, 6.6211007449728205)